# Constraints on light-dependent respiration

Fisher NL, Halsey KH. Mechanisms that increase the growth efficiency of diatoms in low light. *Photosynth Res.* 2016;**129**: 183–197. [doi:10.1007/s11120-016-0282-6](http://doi.org/10.1007/s11120-016-0282-6)

In [1]:
import sys, cobra, math
import pandas as pd
import numpy as np
from cobra import Reaction
from scipy.stats import norm

cobra/__init__.py:33 FutureWarning: Python 2 is reaching end of life (see https://www.python.org/dev/peps/pep-0373/) and many cobra dependencies have already dropped support. At the moment it *should* still work but we will no longer actively maintain Python 2 support.


In [2]:
cobra_config = cobra.Configuration()

cobra_config.lower_bound = -10000.
cobra_config.upper_bound = 10000.

cobra_config.solver = 'glpk'

In [3]:
absorbLL = {'EX_photon410_e': 7732.432316,'EX_photon430_e': 7530.191719,'EX_photon450_e': 6777.205304,'EX_photon470_e': 5420.788645,'EX_photon490_e': 4462.950129,'EX_photon510_e': 3778.897393,'EX_photon530_e': 2988.068035,'EX_photon550_e': 2100.53568,'EX_photon570_e': 1300.702583,'EX_photon590_e': 1075.245665,'EX_photon610_e': 1123.387173,'EX_photon630_e': 1460.196353,'EX_photon650_e': 1580.653401,'EX_photon670_e': 4018.115597,'EX_photon690_e': 2155.737164}
absorbML = {'EX_photon410_e': 4730.075289,'EX_photon430_e': 5817.128965,'EX_photon450_e': 5348.203973,'EX_photon470_e': 4050.000013,'EX_photon490_e': 3464.694801,'EX_photon510_e': 2649.794528,'EX_photon530_e': 1876.490736,'EX_photon550_e': 1334.544022,'EX_photon570_e': 873.4095179,'EX_photon590_e': 740.7816246,'EX_photon610_e': 888.7175101,'EX_photon630_e': 1082.718272,'EX_photon650_e': 1178.924274,'EX_photon670_e': 3322.974688,'EX_photon690_e': 1840.91646}
absorbHL = {'EX_photon410_e': 4299.712311,'EX_photon430_e': 4827.773138,'EX_photon450_e': 4682.224642,'EX_photon470_e': 3843.094021,'EX_photon490_e': 3418.26565,'EX_photon510_e': 2978.620455,'EX_photon530_e': 2481.738844,'EX_photon550_e': 2039.337028,'EX_photon570_e': 1451.311983,'EX_photon590_e': 1255.298421,'EX_photon610_e': 1207.59605,'EX_photon630_e': 1475.056726,'EX_photon650_e': 1549.345022,'EX_photon670_e': 2969.670677,'EX_photon690_e': 1924.104339}
cool_white = {'EX_photon410_e': 0.020538848,'EX_photon430_e': 0.044455157,'EX_photon450_e': 0.032857193,'EX_photon470_e': 0.036698467,'EX_photon490_e': 0.075174979,'EX_photon510_e': 0.040060786,'EX_photon530_e': 0.047049592,'EX_photon550_e': 0.233319199,'EX_photon570_e': 0.070267588,'EX_photon590_e': 0.103796078,'EX_photon610_e': 0.1568836,'EX_photon630_e': 0.0624902,'EX_photon650_e': 0.030806635,'EX_photon670_e': 0.024272336,'EX_photon690_e': 0.021329342}
halogen = {'EX_photon410_e':0.001262457,'EX_photon430_e':0.004585348,'EX_photon450_e':0.008020995,'EX_photon470_e':0.015208428,'EX_photon490_e':0.032867028,'EX_photon510_e':0.049849784,'EX_photon530_e':0.065038153,'EX_photon550_e':0.079970685,'EX_photon570_e':0.093411994,'EX_photon590_e':0.111565142,'EX_photon610_e':0.115420353,'EX_photon630_e':0.117526779,'EX_photon650_e':0.1119338,'EX_photon670_e':0.103590713,'EX_photon690_e':0.089748342}

def calc_photon_abs(I,chla,DW,absorb,relArea):
    conv = (60*60)/1e7 #to mmol gDW-1 h-1
    pfa = {f:I*relArea[f]*absorb[f]*conv*(chla/DW) for f in relArea.keys()}
    return pfa

In [4]:
I = [5,60,200]
absorb = [absorbLL,absorbML,absorbHL]

DOC = [0.26*1e-3,2.43*1e-3,3.95*1e-3]

# 95% CI
lwrGross = [0.009502609,0.12697434,0.38567577]
uprGross = [0.01592601,0.16241050,0.47702915]
lwrNet = [-0.0101236265,0.044636908,0.234284802]
uprNet = [0.0040989012,0.081973226,0.327647747]
lwrDR = [-0.018875372,-0.061322432,-0.156790618]
uprDR = [-0.000167587,-0.021101554,-0.05793793]
lwrCup = [0.004934730,0.07092116,0.30134136]
uprCup = [0.007436942,0.10177330,0.35092754]


D1_m = [6.4751E-09,1.16783E-08,1.36247E-08]
D1_b = [2.35114E-06,4.24045E-06,4.94719E-06]
D1 = [D1_m[i]*I[i]+D1_b[i] for i in range(0,3)]

pgchla = [0.627,0.192,0.164]
mgchla = [chla*1e-9 for chla in pgchla]

pgDW = [22.41006436,16.62883198,17.84261965]
gDW = [DW*1e-12 for DW in pgDW]

cells_mL = [1.43e6,2.21e6,1.56e6]
cells_L = [cells_mL[i]*1e3 for i in range(0,3)]

NGAM = [1.554270008,2.233386198,3.715204852]
MMR = [1.1,5.3,7.9] # umol C (mg Chla h)-1
MMR_se = [0.076,0.039,0.018]
lwrMMR = [MMR[i]-1.96*MMR_se[i] for i in range(0,3)]
uprMMR = [MMR[i]+1.96*MMR_se[i] for i in range(0,3)]
LDR = [1.7,5.6,21.6]
avail_C = [1.541189077,16.62490103,130.4128939]

divide_h = [0.008368152,0.036051307,0.066270096]

mu_d = [0.20,0.85,1.54]
mu_d_se = [0.01/math.sqrt(3),0.03/math.sqrt(3),0.03/math.sqrt(3)]
mu_d_lb = [mu_d[i]-(1.96*mu_d_se[i]) for i in range(0,3)]
mu_d_ub = [mu_d[i]+(1.96*mu_d_se[i]) for i in range(0,3)]
mu_h_lb = [mu/24. for mu in mu_d_lb]
mu_h_ub = [mu/24. for mu in mu_d_ub]
mu_h = [mu/24. for mu in mu_d]

GAM_val = [2698,2217,3669]

# pFBA

In [5]:
ERRORTOL = 1e-8
cobra_config.tolerance = ERRORTOL

LL = cobra.io.read_sbml_model('Models/Thaps_LL_model_n.xml')
ML = cobra.io.read_sbml_model('Models/Thaps_ML_model_n.xml')
HL = cobra.io.read_sbml_model('Models/Thaps_HL_model_n.xml')

models = [LL,ML,HL]

No handlers could be found for logger "cobra.io.sbml"


In [6]:
from cobra import Reaction
gam_rxn = Reaction('GAM')
sink_chryso_c = Reaction('sink_chryso_c')
sink_g3p_h = Reaction('sink_g3p_h')

In [11]:
fluxes = []

for i in range(0,3):
    
    with models[i]:
        
        models[i].add_reactions([sink_chryso_c,sink_g3p_h,gam_rxn])
        models[i].reactions.sink_chryso_c.build_reaction_from_string('chryso_c <-- ')
        models[i].reactions.sink_g3p_h.build_reaction_from_string('g3p_h <-- ')
        models[i].reactions.GAM.build_reaction_from_string('GAM_const_c + {0} atp_c + {0} h2o_c --> {0} adp_c + {0} h_c + {0} pi_c'.format(str(GAM_val[i])))
        gam_met = models[i].metabolites.GAM_const_c
        bof = models[i].reactions.query('bof')[0]
        bof.add_metabolites({gam_met:1})
        
        models[i].reactions.sink_chryso_c.lower_bound = -1*avail_C[i]*divide_h[i]*1e-3*(1./6.)*(mgchla[i]/gDW[i])
        models[i].reactions.sink_g3p_h.lower_bound = -1*avail_C[i]*(1-divide_h[i])*1e-3*(1./3.)*(mgchla[i]/gDW[i])

        # f/2
        no3_mM = 0.250
        pi_mM = 0.050
        so4_mM = 28.8
        sio4h4_mM = 0.106
        cncbl3_mM = 3.69e-7
        co2_mM = 0.0107215
        hco3_mM = 1.72733
        
        models[i].reactions.EX_no3_e.lower_bound = -(no3_mM*mu_h[i])/(cells_L[i]*gDW[i])
        models[i].reactions.EX_pi_e.lower_bound = -(pi_mM*mu_h[i])/(cells_L[i]*gDW[i])
        models[i].reactions.EX_so4_e.lower_bound = -(so4_mM*mu_h[i])/(cells_L[i]*gDW[i])
        models[i].reactions.EX_sio4h4_e.lower_bound = -(sio4h4_mM*mu_h[i])/(cells_L[i]*gDW[i])
        models[i].reactions.EX_cncbl3_e.lower_bound = -(cncbl3_mM*mu_h[i])/(cells_L[i]*gDW[i])
        models[i].reactions.EX_co2_e.lower_bound = -((0.21033224*co2_mM)/(0.000688433+co2_mM))*(mgchla[i]/gDW[i])
        models[i].reactions.EX_hco3_e.lower_bound = -((0.226824637*hco3_mM)/(0.286767672+hco3_mM))*(mgchla[i]/gDW[i])


        pfa = calc_photon_abs(I[i],mgchla[i],gDW[i],absorb[i],cool_white)

        for k in pfa.keys():
            models[i].reactions.get_by_id(k).lower_bound = pfa[k]*-1.
            models[i].reactions.get_by_id(k).upper_bound = pfa[k]*-0.9999

        models[i].reactions.EX_o2_e.lower_bound = -10000.
        models[i].reactions.EX_o2_e.upper_bound = 10000.

        models[i].reactions.PSII_u.reaction = '2.0 h2o_u + {1} h_h + 4.0 p680_exc_u + {2} pq_u + {0} ps2d1_u --> 4.0 h_u + o2_u + 4.0 p680_u + {2} pqh2_u + {0} ps2d1_exc_u'.format(str(D1[i]),str((2 - D1[i])*2),str(2 - D1[i]))
        models[i].reactions.PSII_u.lower_bound = lwrGross[i]*(mgchla[i]/gDW[i])
        models[i].reactions.PSII_u.upper_bound = uprGross[i]*(mgchla[i]/gDW[i])
        
        SEC = models[i].problem.Constraint(
            models[i].metabolites.glyclt_e.elements['C']*models[i].reactions.EX_glyclt_e.flux_expression + models[i].metabolites.fol_e.elements['C']*models[i].reactions.EX_fol_e.flux_expression + models[i].metabolites.co_e.elements['C']*models[i].reactions.EX_co_e.flux_expression + models[i].metabolites.cyan_e.elements['C']*models[i].reactions.EX_cyan_e.flux_expression + models[i].metabolites.cynt_e.elements['C']*models[i].reactions.EX_cynt_e.flux_expression + models[i].metabolites.tcynt_e.elements['C']*models[i].reactions.EX_tcynt_e.flux_expression + models[i].metabolites.lac__D_e.elements['C']*models[i].reactions.EX_lac__D_e.flux_expression + models[i].metabolites.etoh_e.elements['C']*models[i].reactions.EX_etoh_e.flux_expression + models[i].metabolites.urea_e.elements['C']*models[i].reactions.EX_urea_e.flux_expression + models[i].metabolites.dmsp_e.elements['C']*models[i].reactions.EX_dmsp_e.flux_expression + models[i].metabolites.glyb_e.elements['C']*models[i].reactions.EX_glyb_e.flux_expression + models[i].metabolites.Nactaur_e.elements['C']*models[i].reactions.EX_Nactaur_e.flux_expression + models[i].metabolites.frmd_e.elements['C']*models[i].reactions.EX_frmd_e.flux_expression + models[i].metabolites.urate_e.elements['C']*models[i].reactions.EX_urate_e.flux_expression + models[i].metabolites.ac_e.elements['C']*models[i].reactions.EX_ac_e.flux_expression + models[i].metabolites.chol_e.elements['C']*models[i].reactions.EX_chol_e.flux_expression + models[i].metabolites.ura_e.elements['C']*models[i].reactions.EX_ura_e.flux_expression + models[i].metabolites.xan_e.elements['C']*models[i].reactions.EX_xan_e.flux_expression + models[i].metabolites.uacgam_e.elements['C']*models[i].reactions.EX_uacgam_e.flux_expression + models[i].metabolites.glu__L_e.elements['C']*models[i].reactions.EX_glu__L_e.flux_expression + models[i].metabolites.asp__L_e.elements['C']*models[i].reactions.EX_asp__L_e.flux_expression + models[i].metabolites.dhps_e.elements['C']*models[i].reactions.EX_dhps_e.flux_expression + models[i].metabolites.for_e.elements['C']*models[i].reactions.EX_for_e.flux_expression + models[i].metabolites.ile__L_e.elements['C']*models[i].reactions.EX_ile__L_e.flux_expression + models[i].metabolites.leu__L_e.elements['C']*models[i].reactions.EX_leu__L_e.flux_expression + models[i].metabolites.val__L_e.elements['C']*models[i].reactions.EX_val__L_e.flux_expression + models[i].metabolites.asn__L_e.elements['C']*models[i].reactions.EX_asn__L_e.flux_expression + models[i].metabolites.ala__L_e.elements['C']*models[i].reactions.EX_ala__L_e.flux_expression + models[i].metabolites.gln__L_e.elements['C']*models[i].reactions.EX_gln__L_e.flux_expression + models[i].metabolites.his__L_e.elements['C']*models[i].reactions.EX_his__L_e.flux_expression + models[i].metabolites.ser__L_e.elements['C']*models[i].reactions.EX_ser__L_e.flux_expression + models[i].metabolites.thr__L_e.elements['C']*models[i].reactions.EX_thr__L_e.flux_expression + models[i].metabolites.gly_e.elements['C']*models[i].reactions.EX_gly_e.flux_expression + models[i].metabolites.pro__L_e.elements['C']*models[i].reactions.EX_pro__L_e.flux_expression + models[i].metabolites.atp_e.elements['C']*models[i].reactions.EX_atp_e.flux_expression + models[i].metabolites.amp_e.elements['C']*models[i].reactions.EX_amp_e.flux_expression,
            lb=DOC[i]*(mgchla[i]/gDW[i]),
            ub=DOC[i]*(mgchla[i]/gDW[i]))
        
        ldr = models[i].problem.Constraint(
            models[i].reactions.RUBISO_h.flux_expression + models[i].reactions.MEHLER_h.flux_expression + models[i].reactions.PTOX_h.flux_expression + models[i].reactions.GOX_x.flux_expression + models[i].reactions.GOX_m.flux_expression + models[i].reactions.AOX_m.flux_expression + models[i].reactions.CYOO_m.flux_expression,
            lb=LDR[i]*1e-3*(mgchla[i]/gDW[i]),
            ub=10000.)
        
        mmr = models[i].problem.Constraint(
            models[i].reactions.PDH_E1a_m.flux_expression + models[i].reactions.IDH_m.flux_expression + models[i].reactions.ICDH_m.flux_expression + models[i].reactions.OGDH_E1_m.flux_expression,
            lb = lwrMMR[i]*1e-3*(mgchla[i]/gDW[i]),
            ub = uprMMR[i]*1e-3*(mgchla[i]/gDW[i]))
                
        PR = models[i].problem.Constraint(
            models[i].reactions.RUBISO_h.flux_expression - 0.025*models[i].reactions.RUBISC_h.flux_expression,
            lb=0.,
            ub=0.)
        
        EC = models[i].problem.Constraint(
            models[i].reactions.NADHOR_m.flux_expression - 0.0015*models[i].reactions.PSI_u.flux_expression,
            lb=0.,
            ub=0.)
        
        models[i].add_cons_vars([SEC,ldr,mmr]) # Fig 3a
#         models[i].add_cons_vars([SEC,ldr,mmr,PR]) # Fig 3b
#         models[i].add_cons_vars([SEC,ldr,mmr,EC]) # Fig 3c

        models[i].reactions.CEF_h.upper_bound = 0.380324527
        models[i].reactions.ATPM_c.upper_bound = NGAM[i]
    
        models[i].reactions.DM_biomass_c.upper_bound = 10000.
        models[i].reactions.DM_eps_c.upper_bound = 0.
        
        models[i].objective = 'DM_biomass_c'
    
        
        print 'Optimizing...'
        soln = cobra.flux_analysis.parsimonious.pfba(models[i],fraction_of_optimum=1.)
        print soln.status
        print soln.fluxes['DM_biomass_c']*24
        

        for r in soln.fluxes.index:
            if r.startswith('EX_') and abs(soln.fluxes.loc[r]) >= ERRORTOL:
                print r, soln.fluxes.loc[r]
                if soln.fluxes.loc[r] == models[i].reactions.get_by_id(r).lower_bound:
                    print r, 'is limiting'
            elif r.startswith('sink_') and abs(soln.fluxes.loc[r]) >= ERRORTOL:
                print r, soln.fluxes.loc[r]
                if soln.fluxes.loc[r] == models[i].reactions.get_by_id(r).lower_bound:
                    print r, 'is limiting'
            elif r.startswith('DM_') and abs(soln.fluxes.loc[r]) >= ERRORTOL:
                print r, soln.fluxes.loc[r]
            elif soln.fluxes.loc[r] == models[i].reactions.get_by_id(r).upper_bound and soln.fluxes.loc[r] != 0:
                print r, 'is limited'
            elif soln.fluxes.loc[r] == models[i].reactions.get_by_id(r).lower_bound and soln.fluxes.loc[r] < 0:
                print r, 'is limiting'
        
        fluxes.append(soln.fluxes)

unknown metabolite 'GAM_const_c' created
Optimizing...
optimal
0.1992901479750177
EX_co2_e -0.263164475949701
EX_no3_e -0.06282132905539459
EX_btn_e -2.59447703394122e-08
EX_h_e -0.10391727543794062
EX_h2o_e -0.11226457462228723
EX_o2_e 0.3978406686285846
EX_pi_e 0.011978587533159862
EX_so4_e -0.0017137409926234481
EX_mg2_e -0.00026720620328291023
EX_thm_e -2.27717453079257e-06
EX_urea_e 0.0010626216796486708
CEF_h is limited
EX_asp__L_e 0.0015529471614371447
EX_photon410_e -7.998142733442801
EX_photon410_e is limiting
EX_photon430_e -16.858740430461005
EX_photon430_e is limiting
EX_photon450_e -11.214451652830846
EX_photon450_e is limiting
EX_photon470_e -10.018606971655732
EX_photon470_e is limiting
EX_photon490_e -16.89632643517483
EX_photon490_e is limiting
EX_photon510_e -7.623975779971577
EX_photon510_e is limiting
EX_photon530_e -7.080163478355117
EX_photon530_e is limiting
EX_photon550_e -24.681837112223626
EX_photon550_e is limiting
EX_photon570_e -4.60288358600427
EX_photon57

In [12]:
df = pd.concat([fluxes[0],fluxes[1],fluxes[2]], axis=1, sort=True)
df.columns = ['0.2','0.85','1.54']
df.to_csv('../Thaps_draft/update_model/Halsey_SECresp.csv') # Fig 3a
# df.to_csv('../Thaps_draft/update_model/Halsey_SECrespPR.csv') # Fig 3b
# df.to_csv('../Thaps_draft/update_model/Halsey_SECrespEC.csv') # Fig 3c